In [186]:
import numpy as np
import matplotlib as plt
import contextlib

# 제 1고지 미분 자동 계산

###### 1단계 상자로서의 변수
###### 2단계 변수를 낳는 함수
###### 3단계 함수 연결
###### 4단계 수치 미분
###### 5단계 역전파 이론
###### 6단계 수동 역전파 이론
###### 7단계 역전파 자동화
###### 8단계 재귀에서 반복문으로
###### 9단계 함수를 더 편리하게
###### 10단계 테스트

## step 1 상자로서의 변수

In [2]:
# Variable 클래스 구현

class Variable :
    def __init__(self, data) :
        self.data = data

In [3]:


data = np.array(1.0)
x = Variable(data)

In [4]:
print(x.data)

1.0


In [5]:
x.data = np.array(2.0)

In [6]:
print(x.data)

2.0


## step2 변수를 낳는 함수

In [7]:
# Function 클래스 구현
class Function :
    def __call__(self, input) :
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [8]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [9]:
print(y)

In [10]:
print(y.data)

100


In [11]:
# real implementation

class Function :
    
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
        
    def forward(self, x) :
        raise NotImplementedError()

In [12]:
# square implementation

class Square(Function) :
    def forward(self, x) :
        return x ** 2

In [13]:
x = Variable(np.array(100))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
10000


## step 3 함수 연결

In [14]:
## exp 함수 구현

class Exp(Function) :
    def forward(self, x) :
        return np.exp(x)
    

In [15]:
A = Square()
B = Exp()
C = Square()

In [16]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## step4 수치 미분

In [17]:
# 미분 개념
# 중앙 차분

def numerical_diff(f, x, eps = 1e-4) :
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [18]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [19]:
# 합성함수의 미분

def f(x) :
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

In [20]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


In [21]:
# 안 씀
# 근데 오차 커질 수 있다. 자릿수 누락과 함께
# 계산량이 너무 많다.



## step5 역전파 이론

In [22]:
# 손실함수인 y에 대한 각 파라미터의 그래디언트를 구하기 위해서임
# 순전파 & 역전파 
# 먼저 순전파하고 각 함수가 입력변수의 값 기억
# 그 값을 토대로 역전파 수행

# 통상값 (인퍼런스한 값) vs 미분값

## step6 수동 역전파

In [23]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None

In [24]:
# 미분을 계산하는 역전파
# forward 메소드 호출 시 건네받은 Variable 인스턴스 유지

class Function :
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output
    
    def forward(self, x) :
        raise NotImplementedError()
    
    def backward(self, gy) :
        raise NotImplementedError()

In [25]:
## Square 
class Square(Function) :
    def forward(self, x) :
        y = x ** 2 
        return y

    def backward(self, gy) :
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [26]:
## Exp
class Exp(Function) :
    def forward(self, x) :
        y = np.exp(x)
        return y

    def backward(self, gy) :
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
        

In [27]:
# forward

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [28]:
# backward
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


## step7 역전파 자동화

In [29]:
# 변수와 함수의 관계 
# 변수에게 있어 함수는 창조자 혹은 부모입니다.
# 연결을 동적으로 하기
# Define-by-Run
# 그냥 링크드 리스트로 계산 그래프 표현

In [30]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

In [31]:
class Function :
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x) :
        raise NotImplementedError()
    
    def backward(self, gy) :
        raise NotImplementedError()    

In [32]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

AssertionError: 

In [ ]:
# 1. 함수를 가져온다.
# 2. 함수의 입력을 가져온다.
# 3. 함수의 backward 메서드를 호출한다.

In [ ]:
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [ ]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [ ]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

In [ ]:
## backward 메서드 추가하기

In [ ]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        f = self.creator
        if f is not None :
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [ ]:
a = [1,2]
b = [1,2]
a is b

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

## step8 재귀에서 반복문으로

In [ ]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None :
                funcs.append(x.creator)

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

In [ ]:
print(y.creator)
print(b.creator)
print(a.creator)

## step9 함수를 더 편리하게

In [ ]:
# 사용자 편의를 위한 3가지 개선

#### 첫번째 개선 - 일일히 함수 크래스 정의 안하기

In [ ]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

## -> 너무 귀찮아 ㅠㅠ

In [ ]:
def square(x) :
    f = Square()
    return f(x)

def exp(x) :
    f = Exp()
    return f(x)

## -> 이것도 좀 귀찮아 ㅠㅠ    

In [ ]:
def square(x) :
    return Square()(x)

def exp(x) :
    return Exp()(x)

In [ ]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

In [ ]:
# 연속으로 적용해보기
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)

#### 두번째 개선 - backward 메소드 간소화 일일히 맨처음 기울기 1 제외

In [ ]:
class Variable :
    def __init__(self, data) :
        self.data = data 
        self.grad = None
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None :
                funcs.append(x.creator)

In [ ]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

#### 세번째 개선 - ndarray만 취급하기

In [ ]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None :
                funcs.append(x.creator)

In [ ]:
x = Variable(np.array(1.0))
x = Variable(None)

x = Variable(1)

In [ ]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

In [ ]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

In [4]:
def as_array(x) :
    if np.isscalar(x) :
        return np.array(x)
    return x

In [ ]:
import numpy as np
np.isscalar(np.float64(1.0))

In [ ]:
np.isscalar(2.0)

In [ ]:
np.isscalar(np.array(1.0))

In [ ]:
np.isscalar(np.array([1,2,3]))

In [ ]:
class Function :
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x) :
        raise NotImplementedError()
    
    def backward(self, gy) :
        raise NotImplementedError()    

## step10 테스트

In [ ]:
def numerical_diff(f, x, eps=1e-4) :
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [ ]:
import unittest

class SquareTest(unittest.TestCase) :
    def test_forward(self) :
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self) :
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self) :
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
        

unittest.main(argv=[''], exit=False)

In [ ]:
## np.allclose는 아래와 같다.
## |a - b| <= (atol + rtol * |b|)

## 칼럼: 자동 미분

In [ ]:
# 미분 계산 3가지
## 수치 미분
## 기호 미분
## 자동 미분 - forward 모드 & reverse 모드 

# 제 2고지 자연스러운 코드로

###### 11단계 가변 길이 인수(순전파 편)
###### 12단계 가변 길이 인수(개선 편)
###### 13단계 가변 길이 인수(역전파 편)
###### 14단계 같은 변수 반복 사용
###### 15단계 복잡한 계산 그래프(이론편)
###### 16단계 복잡한 계산 그래프(구현편)
###### 17단계 메모리 관리와 순환 참조
###### 18단계 메모리 절약 모드
###### 19단계 변수 사용성 개선
###### 20단계 연산자 오버로드(1)
###### 21단계 연산자 오버로드(2)
###### 22단계 연산자 오버로드(3)
###### 23단계 패키지로 정의
###### 24단계 복잡한 함수의 미분

## step11 가변 길이 인수(순전파 편)

In [ ]:
class Function :
    def __call__(self, inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs) :
        raise NotImplementedError()

    def backward(self, gys) :
        raise NotImplementedError()


In [ ]:
class Add(Function) :
    def forward(self, xs) :
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [ ]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

## step12 가변 길이 인수(개선 편)

In [ ]:
# 사용하는 사람과 구현하는 사람의 위한 개선

In [5]:
class Function :
    def __call__(self, *inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs) :
        raise NotImplementedError()

    def backward(self, gys) :
        raise NotImplementedError()


In [6]:
def f(*x) :
    print(x)

In [7]:
f(1,3,3,44444)

(1, 3, 3, 44444)


In [8]:
class Add(Function) :
    def forward(self, xs) :
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [9]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


In [10]:
class Function :
    def __call__(self, *inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple) :
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs 
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs) :
        raise NotImplementedError()

    def backward(self, gys) :
        raise NotImplementedError()

In [11]:
class Add(Function) :
    def forward(self, x0, x1) :
        y = x0 + x1
        return y

In [12]:
def add(x0, x1) :
    return Add()(x0, x1)

In [13]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## step13 가변 길이 인수(역전파 편)

In [134]:
class Add(Function) :
    def forward(self, x0, x1) :
        y = x0 + x1
        return y

    def backward(self, gy) :
        return gy, gy

In [135]:
def add(x0, x1) :
    return Add()(x0, x1)

In [16]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None :
                funcs.append(x.creator)

In [17]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                x.grad = gx

                if x.creator is not None :
                    funcs.append(x.creator)

In [136]:
class Square(Function) :
    def forward(self, x) :
        y = x ** 2
        return y
        
    def backward(self, gy) :
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    

In [137]:
def square(x) :
    return Square()(x)

In [20]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


## step14 같은 변수 반복 사용

In [ ]:
# 같은 변수 사용으로 미분값 덮어 씌어짐

In [ ]:
x = Variable(np.array(3.0))
y = add(x, x)
print("y", y.data)

y.backward()
print("x.grad", x.grad)

In [ ]:
# 잘못된 점

class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                x.grad = gx   # 잘못된 점

                if x.creator is not None :
                    funcs.append(x.creator)

In [2]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                if x.grad is None :
                    x.grad = gx
                else :
                    x.grad = x.grad + gx
                    
                if x.creator is not None :
                    funcs.append(x.creator)

In [ ]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

In [ ]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)

In [ ]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

y = add(add(x, x), x)
y.backward()
print(x.grad)

In [3]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                if x.grad is None :
                    x.grad = gx
                else :
                    x.grad = x.grad + gx
                    
                if x.creator is not None :
                    funcs.append(x.creator)

    def cleargrad(self) :
        self.grad = None
        

In [ ]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.cleargrad()
y = add(add(x, x), x)
y.backward()
print(x.grad)

## step15 복잡한 계산 그래프(이론 편)

In [ ]:
# 함수 우선 순위
# 함수와 변수의 세대를 기록
# 

## step16 복잡한 계산 그래프(구현 편)

In [2]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None
        self.generation = 0

    def set_creator(self, func) :
        self.creator = func
        self.generation = func.generation + 1

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                if x.grad is None :
                    x.grad = gx
                else :
                    x.grad = x.grad + gx
                    
                if x.creator is not None :
                    funcs.append(x.creator)

    def cleargrad(self) :
        self.grad = None

In [141]:
def as_array(x) :
    if np.isscalar(x) :
        return np.array(x)
    return x



class Function(object) :
    def __call__(self, *inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple) :
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs 
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs) :
        raise NotImplementedError()

    def backward(self, gys) :
        raise NotImplementedError()

In [140]:
generations = [2,0,1,4,2]
funcs = []

for g in generations :
    f = Function()
    f.generation = g
    funcs.append(f)

[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [26]:
funcs.sort(key=lambda x : x.generation)
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [27]:
f = funcs.pop()
f.generation

4

In [5]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None
        self.generation = 0

    def set_creator(self, func) :
        self.creator = func
        self.generation = func.generation + 1

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f) :
            if f not in seen_set :
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x : x.generation)

        add_func(self.creator)
        
        while funcs :
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                if x.grad is None :
                    x.grad = gx
                else :
                    x.grad = x.grad + gx
                    
                if x.creator is not None :
                    add_func(x.creator)
                
    def cleargrad(self) :
        self.grad = None

In [187]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


## step17 메모리 관리와 순환 참조

In [8]:
# 파이썬의 메모리 관리 설명 (Cpython)

# 메모리 관리
## 메모리 누수
## 2메모리 부족

# 파이썬 
## 참조수 세기 "참조 카운트"
## 세대를 기준으로 쓸모없어진 객체를 회수하는 방식 "GC" (Garbage Collection)

In [9]:
## 참조 카운트 방식의 메모리 관리

### 모든 객체는 카운트가 0인 상태로 생성되고, 
### 다른 객체가 참조할 때마다 1씩 증가
### 반대로 객체에 대한 참조가 끊길 때마다 1만큼 감소하다가 0이 되면 
### 파이썬 인터프리터가 회수
### 객체가 더이상 필요 없어지면 즉시 메모리에서 삭제

#### - 대입 연산자를 사용할 때
#### - 함수에 인수로 전달할 때
#### - 컨테이너 타입 객체(리스트, 튜플 클래스 등)에 추가할 떄


In [33]:
class obj :
     pass

def f(x) :
    print(x)

a = obj()
f(a)

a = None

In [42]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c

a = b = c = None

In [43]:
x = Variable(np.array(2.0))

In [44]:
x.__dict__

{'data': array(2.), 'grad': None, 'creator': None, 'generation': 0}

In [ ]:
## 순환 참조

In [47]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
c.a = a

a = b = c = None

In [ ]:
## weakref 모듈

In [95]:
import weakref
import numpy as np

In [96]:
a = np.array([1,2,3])
bb = weakref.ref(a)

In [97]:
bb

<weakref at 0x00000191ED9046D0; to 'numpy.ndarray' at 0x00000191ED962BB0>

In [98]:
bb()

array([1, 2, 3])

In [99]:
a = None

In [102]:
import gc
gc.collect()

bb

<weakref at 0x00000191ED9046D0; to 'numpy.ndarray' at 0x00000191ED962BB0>

In [103]:
a = np.array([1,2,3])
b = weakref.ref(a)
print(b)
print(b())
a = None
print(b)
print(b())

<weakref at 0x00000191ED991620; to 'numpy.ndarray' at 0x00000191ED961B90>
[1 2 3]
<weakref at 0x00000191ED991620; dead>
None


In [17]:
import weakref

In [133]:
def as_array(x) :
    if np.isscalar(x) :
        return np.array(x)
    return x



class Function(object) :
    def __call__(self, *inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple) :
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs 
        self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs) :
        raise NotImplementedError()

    def backward(self, gys) :
        raise NotImplementedError()

In [11]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None
        self.generation = 0

    def set_creator(self, func) :
        self.creator = func
        self.generation = func.generation + 1

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f) :
            if f not in seen_set :
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x : x.generation)

        add_func(self.creator)
        
        while funcs :
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                if x.grad is None :
                    x.grad = gx
                else :
                    x.grad = x.grad + gx
                    
                if x.creator is not None :
                    add_func(x.creator)
                
    def cleargrad(self) :
        self.grad = None

In [ ]:
for i in range(10) :
    x = Variable(np.random.randn(10000))
    y = square(square(square(x)))

## step18 메모리 절약 모드

In [ ]:
# 역전파 시 불필요한 미분 결과 보관하지 않기
# 역전파가 필요 없는 경우용 모드

In [129]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 None
2.0 1.0


In [142]:
# 불필요한 중간변수의 미분값 제거하기

class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        
        self.data = data
        self.grad = None 
        self.creator = None
        self.generation = 0

    def set_creator(self, func) :
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self) :
        self.grad = None

    def backward(self, retain_grad = False) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f) :
            if f not in seen_set :
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x : x.generation)

        add_func(self.creator)
        
        while funcs :
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple) :
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) :
                if x.grad is None :
                    x.grad = gx
                else :
                    x.grad = x.grad + gx
                    
                if x.creator is not None :
                    add_func(x.creator)
                    
            if not retain_grad :
                for y in f.outputs :
                    y().grad = None

In [143]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [ ]:
## 순전파만 할 경우

In [144]:
class Config :
    enable_backprop = True

In [145]:
def as_array(x) :
    if np.isscalar(x) :
        return np.array(x)
    return x



class Function(object) :
    def __call__(self, *inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple) :
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop :
            self.generation = max([x.generation for x in inputs])
            for output in outputs :
                output.set_creator(self)
            self.inputs = inputs 
            self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs) :
        raise NotImplementedError()

    def backward(self, gys) :
        raise NotImplementedError()

In [147]:
Config.enable_back_prop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()

Config.enable_back_prop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))

In [148]:
## 모드 전환 쉽게 만들기 
'''
with using_config('enable_back_prop', False) :
    x = Variable(np.array(2.0))
    y = square(x)
'''

"\nwith using_config('enable_back_prop', False) :\n    x = Variable(np.array(2.0))\n    y = square(x)\n"

In [188]:
import contextlib

@contextlib.contextmanager
def config_test() :
    print("start")
    try :
        yield 
    finally :
        print("done")
        
with config_test() :
    print("process ...")

start
process ...
done


In [189]:
import contextlib

@contextlib.contextmanager
def using_config(name, value) :
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try :
        yield
    finally :
        setattr(Config, name, old_value)

In [190]:
with using_config('enable_backprop', False) :
    x = Variable(np.array(2.0))
    y = square(x)

In [200]:
def no_grad() :
    return using_config('enable_backprop', False)

with no_grad() :
    x = Variable(np.array(2.0))
    y = square(x)

## step19 변수 사용성 개선

## step20 연산자 오버로드(1)

## step21 연산자 오버로드(2)

## step22 연산자 오버로드(3)

## step23 패키지로 정리

## step24 복잡한 함수의 미분